# Afiniczne Ciągłe Automaty Komórkowe (ACCAs)

Wykład: https://github.com/houp/ca-class/blob/main/slides/lecture7.pdf

## Importy

In [1]:
import math
import random
import sympy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

## zad.13
Napisz program, który dla zadanego LUT (binarnego) oraz cLUT (real-valued) wygeneruje wielomian odpowiadający CA / ACCA.

Wersja dla normalnych ludzi: zakładamy, że promień sąsiedztwa = 1.

Wersja z (*): piszemy ogólny program dla dowolnego promienia sąsiedztwa.

Tak czy inaczej do napisania jest funkcja:\
**get_ca_polynomial(lut)** -> polynomial

Użyj SymPy lub podobnej biliboteki do operacji na wielomianach.

In [ ]:
# def get_ca_polynomial(lut: np.ndarray):
    
#     return polynomial

In [2]:
x, y = sp.symbols('x y')
polynomial = 2*x*(1-y)
print(polynomial)

2*x*(1 - y)


## zad.14
Korzystając z rozwiązania Zadania 13 oraz z wcześniejszych programów, napisz funkcję, która dla zadanego cLUT i zadanej konfiguracji początkowej generuje space-time diagram ACCA.

**acca_space_time(clut, initial_conf, step)** -> space time diagram w matplotlib

In [ ]:
# traffic = wolfram_rule_to_lut(184)
# majority = wolfram_rule_to_lut(232)

# alpha=0.9
# traffic_majority_clut = alpha * traffic + (1-alpha) * majority
# conf = np.random.randint(0,2,49)
# acca_space_time(traffic_majority_clut, conf, 49)